In [1]:
# ----------------------------------------------------------------------------------- #
# SON LOS IMPORTS DEL LAB ANTERIOR, LOS DESCOMENTAMOS A MEDIDA QUE LOS VAYAMOS USANDO #
# ----------------------------------------------------------------------------------- #

# from collections import defaultdict
from array import array
import nltk
#nltk.download('stopwords')
# # nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
import math
import numpy as np
import collections
from numpy import linalg as la
# import json
# import random
# import re
import pandas as pd
import time

### Loading the processed tweets

In [2]:
output_folder = '../output/'
data = pd.read_csv(output_folder + "lab1_tweets_df.csv", sep='|')
data.head(2)

,Document,Id,Text,Username,Date,Hashtags,Mentions,Likes,Retweets,Url,Clean_text
0,doc_1,1575918182698979328,So this will keep spinning over us until 7 pm…...,suzjdean,Fri Sep 30 18:39:08 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/suzjdean/status/1575918182...,"['keep', 'spin', 'us', 'away', 'alreadi', 'hur..."
1,doc_2,1575918151862304768,Our hearts go out to all those affected by #Hu...,lytx,Fri Sep 30 18:39:01 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/lytx/status/15759181518623...,"['heart', 'go', 'affect', 'hurricaneian', 'wis..."


# Indexing

## 1. Build inverted index

In [3]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line=  line.lower() ## Transform in lowercase
    line=  line.split() ## Tokenize the text to get a list of terms
    line= [x for x in line if x not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemmer.stem(x) for x in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [4]:
def create_index_tfidf(dataframe):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    dataframe -- DataFrame containing tweet information
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    num_documents = dataframe.shape[0]
    index = defaultdict(list)
    tf = defaultdict(list)  # term frequencies of terms in documents (documents in the same order as in the main index)
    df = defaultdict(int)   # document frequencies of terms in the corpus
    idf = defaultdict(float)
    for row in dataframe.iterrows():
        doc_id = row[1]['Document']
        terms = row[1]['Clean_text']  
        terms = eval(terms)
        
        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[doc_id, [position]]
                
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        # calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] = df[term]+1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)

    return index, tf, df, idf


In [5]:
start_time = time.time()
new_index, tf, df, idf = create_index_tfidf(data)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 299.84 seconds


### Words analysis in the collection:

In [29]:
## ============================= ##
## show top DF in the collection ##
## ============================= ##

word = []
word_df = []
for item in df.items():
    word.append(item[0])
    word_df.append(item[1])

document_frequency_df = pd.DataFrame(list(zip(word,word_df)), columns=['Word', 'DF'])
document_frequency_df.sort_values(by='DF', ascending=False).head(20)

,Word,DF
5,hurricaneian,3988
72,florida,881
47,hurrican,793
70,ian,781
211,help,386
122,amp,362
25,storm,352
87,carolina,297
303,flood,289
293,power,275


## 2. Test queries

In [6]:
queries = [
    "Floodings in South Carolina", 
    "HuracaineIan disaster", 
    "Damage of HuracaineIan", 
    "Florida floodings", 
    "Storm and wind in Florida"
]


## 3. Ranking

### Create our grand trouth

In [7]:
def rank_documents(terms, docs, index, idf, tf):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    
    Returns:
    list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]=query_terms_count[term]/query_norm *idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]
    
    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(doc_scores) == 0:
        print("No results found for query")
    #print ('\n'.join(result_docs), '\n')
    return doc_scores

In [8]:
def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf)
    return ranked_docs

In [9]:
def search_tf_idf_subset(query, index, subset):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term" and are in the subset.                       
            term_docs=[posting[0] for posting in index[term] if posting[0] in subset]
            
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf)
    return ranked_docs

In [43]:
queries = queries
top = 2
ratings = dict()
relevance = dict()

query_id = 0
for query in queries:
    ranked_docs = search_tf_idf(queries[query_id], new_index)
    for doc in ranked_docs[:top]:
        print("==========")
        print("Query: ",query)
        print("Is this document relevant to the query? 0 for no, 1 for yes:")
        print(data[data['Document']==doc[1]]['Text'].iloc[0])
        print("==========")
        rate = input()
        rate = float(rate)
        ratings[(doc[1],query_id)] = rate
        relevance[(doc[1],query_id)] = doc[0]
    print(ratings)
    query_id += 1

Query:  Floodings in South Carolina
Is this document relevant to the query? 0 for no, 1 for yes:
South Carolina #HurricaneIan https://t.co/yTA4dFUC2V
Query:  Floodings in South Carolina
Is this document relevant to the query? 0 for no, 1 for yes:
Flooding in Garden City, South Carolina 

#GardenCity #SouthCarolina #HurricaneIan https://t.co/5W328kjWkH


ValueError: could not convert string to float: ''

In [ ]:
doc_id = [key[0] for key in ratings.keys()]
query_id = [key[1] for key in ratings.keys()]
rating = [value for value in ratings.values()]
predicted_relevance = [value for value in relevance.values()]
our_truths = pd.DataFrame()
our_truths['doc_id'] = doc_id
our_truths['query_id'] = query_id
our_truths['rating'] = rating
our_truths['predicted_relevance'] = predicted_relevance
our_truths

In [ ]:
one = our_truths.to_dict()
#imprimir y tenerlo por ahí
two = pd.DataFrame(one)

In [34]:
data.loc[data['Document'] == 'doc_254']['Text'].values[0]

'South Carolina #HurricaneIan https://t.co/yTA4dFUC2V'

In [36]:
for query in queries:
    ranked_docs = search_tf_idf(query, new_index)
    top = 10

    print("\n======================\nTop {} results out of {} for the searched query '{}':\n".format(top, len(ranked_docs),query))
    for doc in ranked_docs[:top]:
        print("{} - score {} Text:".format(doc[1], doc[0]))
        print("\t{}".format(data.loc[data['Document'] == doc[1]]['Text'].values[0]))


Top 10 results out of 605 for the searched query 'Floodings in South Carolina':

doc_254 - score 4.74475387696339 Text:
	South Carolina #HurricaneIan https://t.co/yTA4dFUC2V
doc_1289 - score 4.315205653041319 Text:
	Flooding in Garden City, South Carolina 

#GardenCity #SouthCarolina #HurricaneIan https://t.co/5W328kjWkH
doc_2874 - score 4.10872348195652 Text:
	From .@ABCNews4 in Charleston, South Carolina - #HurricaneIan https://t.co/TLCRoQ3VvD
doc_2834 - score 4.10872348195652 Text:
	from .@ABCNews4 in Charleston, South Carolina - #HurricaneIan https://t.co/nbxQROQSKW
doc_174 - score 4.10872348195652 Text:
	South Carolina #HurricaneIan here we go
doc_2484 - score 4.067471844340135 Text:
	Floods trap many in Florida as Ian heads to South Carolina
https://t.co/zmUfHC9egM #HurricaneIan
doc_493 - score 3.962766890072915 Text:
	Just south of Myrtle Beach in South Carolina. #HurricaneIan #Ian #ScWx https://t.co/ErHr5X5cQQ
doc_2505 - score 3.674842282261912 Text:
	⚠️ #HurricaneIan about to

# Evaluation

## 1. Baseline

Evaluation of the ranking considering only the subset of documents in the ground truth file provided ("evaluation_gt.csv")

In [11]:
ground_truth = pd.read_csv('evaluation_gt.csv')
ground_truth.head()

,doc,query_id,label
0,doc_12,1,1
1,doc_9,1,1
2,doc_18,1,1
3,doc_45,1,1
4,doc_501,1,1


In [ ]:
def precision_at_k(doc_score, k=10): #binary relevance, predicted relevance, k for a given query
    """
    Parameters
    ----------
    doc_score: Ordered ground truth (true relevance labels).
    k : number of doc to consider.

    Returns
    -------
    precision @k : float

    """
    doc_score = doc_score[:k]
    relevant = sum(doc_score == 1) #get number of relevant documents
    return float(relevant) / k #calculae precision at k, which is the number of relevant documents trieved at k

In [39]:
def recall_at_k(doc_score, k=10): #binary relevance, predicted relevance, k for a given query
    """
    Parameters
    ----------
    doc_score: Ordered ground truth (true relevance labels).
    k : number of doc to consider.

    Returns
    -------
    precision @k : float

    """
    total_relevant = sum(doc_score == 1)
    doc_score = doc_score[:k]
    relevant = sum(doc_score == 1) #get number of relevant documents
    return float(relevant) / total_relevant #calculae precision at k, which is the number of relevant documents trieved at k

In [ ]:
def avg_precision_at_k(doc_score, k=10): #binary relevance, predicted relevance, k for a given query
    """
    Parameters
    ----------
    doc_score: Ordered ground truth (true relevance labels).
    k : number of doc to consider.

    Returns
    -------
    average precision @k : float
    """
    gtp = np.sum(doc_score == 1) #Total number of gt positives
    doc_score = doc_score[:k] #same as for precision
    ## if all documents are not relevant
    if gtp == 0:
        return 0
    n_relevant_at_i = 0
    prec_at_i = 0
    for i in range(len(doc_score)):
        if doc_score[i] == 1: #only add the P@k when the doc is relevant
            n_relevant_at_i += 1
            prec_at_i += n_relevant_at_i / (i + 1) #calculate P@K (#docs relevant at k/k)
    return prec_at_i / gtp #return ap

In [ ]:
def f1_score():

In [44]:
def ndcg(queries, doc_score):

    ndcg_scores = []

    for query in queries:
        ncdg_score = 0
        for i, score in enumerate(doc_score):
            ncdg_score += score/math.log(2,i+1)

        ndcg_scores.append(ncdg_score)

    for i, ndcg in enumerate(ndcg_scores):
        top = max(ndcg_scores)
        ndcg_scores[i] = ndcg/top

    return ndcg_scores


In [ ]:
def map_at_k(search_res, k=10): #receives all the search esults dataframe containing all the queries and the results and relevances
    """
    Parameters
    ----------
    search_res: search results dataset containing:
        query_id: query id.
        doc: document id.
        predicted_relevance: relevance predicted through LightGBM.
        label: actual score of the document for the query (ground truth).

    Returns
    -------
    mean average precision @ k : float
    """
    avp = []
    for q in search_res["query_id"].unique():  # loop over all query ids
        curr_data = search_res[search_res["query_id"] == q]  # select data for current query (get a slice of the dataframe keeping only the data related to the current query)
        avp.append(avg_precision_at_k(np.array(curr_data["label"]), k))  #append average precision for current query
    return np.sum(avp) / len(avp), avp  # return mean average precision

In [ ]:
def rr_at_k(doc_score, k=10):
    """
    Parameters
    ----------
    doc_score: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.

    Returns
    -------
    Reciprocal Rank for qurrent query
    """

    doc_score = doc_score[:k]
    if np.sum(doc_score) == 0:  # if there are not relevant doument return 0
        return 0
    return 1 / (np.argmax(doc_score == 1) + 1)  # hint: to get the position of the first relevant document use "np.argmax" (+1 because the idex starts from 0)


In [41]:
results = pd.DataFrame()

ground_truth_queries = {
    1:"Landfall in South Carolina",
    2:"Help and recovery during the hurricanee disaster",
    3:"Floodings in South Carolina"}

final = pd.DataFrame()
for query_id, query in ground_truth_queries.items():
    ranking = search_tf_idf_subset(query, new_index, set(ground_truth[ground_truth['query_id']==query_id]['doc']))
    print("\n======================\nResults for the subset of docs in the ground truth query '{}':\n".format(query))
    rankingdf = pd.DataFrame(ranking, columns=['predicted_relevance', 'doc']).merge(ground_truth[ground_truth['query_id']==query_id], how='outer', on='doc').fillna(0)
    display(rankingdf)
    print("P@K",precision_at_k(rankingdf['label']))
    print("AP@K",avg_precision_at_k(rankingdf['label']))
    print("R@K", recall_at_k(rankingdf['label']))
    print("RR@K",rr_at_k(rankingdf['label']))
    if final.empty:
        final = rankingdf
    else:
        final = pd.concat([final, rankingdf])

print("MAP@K",map_at_k(final))
         



Results for the subset of docs in the ground truth query 'Landfall in South Carolina':



,predicted_relevance,doc,query_id,label
0,3.835268,doc_82,1,1
1,3.672444,doc_501,1,1
2,2.918111,doc_165,1,1
3,2.600135,doc_12,1,1
4,2.426312,doc_100,1,1
5,2.247656,doc_18,1,1
6,1.500909,doc_9,1,1
7,1.500909,doc_122,1,1
8,1.203699,doc_45,1,1
9,1.162720,doc_52,1,1


P@K 1.0
AP@K 1.0
R@K 1.0
RR@K 1.0

Results for the subset of docs in the ground truth query 'Help and recovery during the hurricanee disaster':



,predicted_relevance,doc,query_id,label
0,3.105629,doc_402,2,1
1,2.476477,doc_268,2,1
2,2.262324,doc_504,2,1
3,1.222461,doc_321,2,1
4,0.987710,doc_1233,2,0
5,0.758298,doc_373,2,1
6,0.683397,doc_158,2,1
7,0.596469,doc_358,2,1
8,0.596469,doc_175,2,1
9,0.516648,doc_453,2,1


P@K 0.9
AP@K 0.8354365079365079
R@K 0.9
RR@K 1.0

Results for the subset of docs in the ground truth query 'Floodings in South Carolina':



,predicted_relevance,doc,query_id,label
0,3.385289,doc_66,3,1
1,3.116143,doc_148,3,1
2,2.728733,doc_65,3,1
3,2.728733,doc_65,3,1
4,2.191773,doc_30,3,1
5,2.191773,doc_30,3,1
6,2.108991,doc_198,3,1
7,1.764199,doc_370,3,1
8,0.914432,doc_1195,3,0
9,0.849855,doc_150,3,1


P@K 0.9
AP@K 0.89
R@K 0.9
RR@K 1.0
MAP@K (0.908478835978836, [1.0, 0.8354365079365079, 0.89])


In [20]:
from sklearn.manifold import TSNE

vectors = #falta meter la data

plot = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(vectors)